# Actual Recommendation

본 프로젝트의 목적은 RMSE를 계산하는 것이 아니라,   
실제 추천을 구현하는 것이기 때문에,   
추가적으로 추천의 결과물을 시각화하는 자료를 생성했습니다.

In [1]:
import numpy as np
import pandas as pd
## 아래 얘는 따로 놀긴 하는데, 위에 애들이 반환하는 csr matrix로 갖고 논다

from data_utils import *
from preprocessing import *
from util_functions import *
from prediction_file import *
from model import *
from train import *
from torch import torch
import multiprocessing as mp
import warnings
warnings.filterwarnings(action='ignore')

import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, Conv1d
from torch_geometric.nn import GCNConv, RGCNConv, global_sort_pool, global_add_pool
from torch_geometric.utils import dropout_adj
from util_functions import *
import pdb
import time
pd.set_option('display.max_row', 500)

data = pd.read_csv("C:\\Users\\kswoo\\OneDrive\\바탕 화면\\IGMC\\ml-1m\\new_ratings.csv")
data.head(10)

,userId,movieId,ratings,zip_code
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [2]:
user_feature = pd.read_csv('C:\\Users\\kswoo\\OneDrive\\바탕 화면\\IGMC\\ml-1m\\users.dat', sep = "::", 
                          names = ['userId', 'sex', 'age', 'occupation', 'zipcode'])
movie_feature = pd.read_csv('C:\\Users\\kswoo\\OneDrive\\바탕 화면\\IGMC\\ml-1m\\movies.dat', sep = "::", 
                           names = ['movieId', 'name', 'genre'])

# 추천 모델 선언
recommender = IGMC_rec_interface(data, movie_feature)

In [20]:
# 전체영화 
print(movie_feature.shape)
movie_feature.head(10)

(3883, 3)


,movieId,name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [18]:
recommender.check_user_preference(200)

,movieId,name,genre,ratings
7,8,Tom and Huck (1995),Adventure|Children's,3
168,170,Hackers (1995),Action|Crime|Thriller,5
928,940,"Adventures of Robin Hood, The (1938)",Action|Adventure,4
1045,1059,William Shakespeare's Romeo and Juliet (1996),Drama|Romance,5
1111,1127,"Abyss, The (1989)",Action|Adventure|Sci-Fi|Thriller,4
1416,1441,Benny & Joon (1993),Comedy|Romance,4
1564,1605,Excess Baggage (1997),Adventure|Romance,1
1842,1911,Doctor Dolittle (1998),Comedy,2
1972,2041,Condorman (1981),Action|Adventure|Children's|Comedy,5
2286,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy,3


In [11]:
dir_ = "C:\\Users\\kswoo\\OneDrive\\바탕 화면\\IGMC\\processing\\processing\\model_save\\"
model = IGMC(recommender.initializer, 
            latent_dim = [32, 32, 32, 32], # Layer Feature의 차원
            num_relations = 5,
            num_bases = 3, 
            regression = True, 
            adj_dropout = 0
            )
model.load_state_dict(torch.load(os.path.join(dir_, 'model_checkpoint16.pth')))
model.to(device)
model.eval()

recommender.recommend(200, 10, model)

100%|██████████████████████████████████████████████████████████████████████████████| 3706/3706 [01:05<00:00, 56.80it/s]

추천이 끝났습니다! 200번 유저의 추천 결과물은 아래와 같습니다.


,movieId,name,genre
39,40,"Cry, the Beloved Country (1995)",Drama
315,318,"Shawshank Redemption, The (1994)",Drama
857,868,Death in Brunswick (1991),Comedy
1473,1507,Paradise Road (1997),Drama|War
1597,1643,"Mrs. Brown (Her Majesty, Mrs. Brown) (1997)",Drama|Romance
2116,2185,I Confess (1953),Thriller
2123,2192,See the Sea (Regarde la mer) (1997),Drama
2152,2221,Blackmail (1929),Thriller
2540,2609,"King of Masks, The (Bian Lian) (1996)",Drama
2764,2833,Lucie Aubrac (1997),Romance|War
